In [1]:
import pandas as pd 
from pandas.io.json import json_normalize
from pathlib import Path

p = Path('.').cwd()
raw_data = p / 'raw_data'
treated_data = p / 'treated_data'

def max_len_on(df, field):
    return df[field].map(str).apply(len).max()

# Cleaning Datasets

## GS1 

In [2]:
    gs1 = pd.read_json(raw_data / 'gs1.jl', lines=True)
    gs1_responses = pd.io.json.json_normalize(gs1.response)
    gtins = pd.merge(gs1, gs1_responses, left_index=True, right_index=True)    
    
    gtins = gtins.rename(columns={'cnpj_manufacturer':'cnpj',
                                  'gepirParty.partyDataLine.address.city':'city',
                                  'gepirParty.partyDataLine.address.state':'state'})
    
    max_gtin = max_len_on(gtins, 'gtin')
    max_cnpj = max_len_on(gtins, 'cnpj')

    gtins = gtins[(gtins['status'] == 'OK') & (gtins['cnpj'].notnull())]

    gtins['gtin'] = gtins['gtin'].map(str).apply(lambda x: x.zfill(max_gtin))
    gtins['cnpj'] = gtins['cnpj'].map(int).map(str).apply(lambda x: x.zfill(max_gtin))

    gtins[['cnpj','gtin', 'city', 'state']].to_csv(treated_data / 'gs1_treated.csv', index=False)

## Infomix

In [3]:
    infomix = pd.read_csv(raw_data / 'infomix.tsv', sep='\t')

    max_cnpj = max_len_on(infomix, 'cnpj')
    max_gtin = max_len_on(infomix, 'gtin')

    infomix['category'] = infomix.category.str.upper()
    infomix['cnpj'] = infomix['cnpj'].map(str).apply(lambda x: x.zfill(max_cnpj))
    infomix['gtin'] = infomix['gtin'].map(str).apply(lambda x: x.zfill(max_gtin))

    infomix.to_csv(treated_data / 'infomix_treated.csv', index=False)


## Cosmos

In [4]:
    cosmos = pd.read_json(raw_data / 'cosmos.jl', lines=True)
    cosmos_responses = pd.io.json.json_normalize(cosmos.response)
    cosmos_full = pd.merge(cosmos, cosmos_responses, left_index=True, right_index=True) 

    cosmos_full = cosmos_full[cosmos_full['status'] == 'OK']
    cosmos_full = cosmos_full[['gtin_y','description']].rename(columns={'gtin_y':'gtin'})
    
    cosmos_full.to_csv(treated_data / 'cosmos_treated.csv',index=False)

In [5]:
cosmos_responses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20588 entries, 0 to 20587
Data columns (total 11 columns):
autited            9676 non-null object
brand              4349 non-null object
description        9676 non-null object
gcp.code           2392 non-null object
gcp.description    2392 non-null object
gtin               9713 non-null object
image              8245 non-null object
ncm.code           8896 non-null object
ncm.description    8896 non-null object
status             20588 non-null object
units              9713 non-null object
dtypes: object(11)
memory usage: 1.7+ MB


## CNPJs

In [6]:
    cnpjs = pd.read_json(raw_data / 'cnpjs_receita_federal.jl', lines=True)
    cnpjs_responses = pd.io.json.json_normalize(cnpjs.response)
    cnpjs_full = pd.merge(cnpjs, cnpjs_responses, left_index=True, right_index=True) 
    
    cnpjs_full = cnpjs_full[cnpjs_full['situacao'] == 'ATIVA']
    cnpjs_full = cnpjs_full[['nome','cnpj_x','municipio','uf']].rename(columns={'cnpj_x':'cnpj', 'uf':'state', 'municipio':'city', 'nome':'name'})

    max_cnpj = max_len_on(cnpjs_full, 'cnpj')
    
    cnpjs_full['cnpj'] = cnpjs_full['cnpj'].map(int).map(str).apply(lambda x: x.zfill(max_cnpj))

    cnpjs_full.to_csv(treated_data / 'cnpjs_treated.csv', index=False)
    
    cnpjs_full.head()

,name,cnpj,city,state
0,FROOTY COMERCIO E INDUSTRIA DE ALIMENTOS S.A.,68093095000179,ATIBAIA,SP
1,ESTRELLA DE GALICIA IMPORTACAO E COMERCIALIZAC...,13492669000190,SAO PAULO,SP
2,DROGARIA NOVA SABARA LTDA,74683392000177,SAO PAULO,SP
3,DJ INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,12423658000195,CARPINA,PE
4,FUGINI ALIMENTOS LTDA,00588458000103,MONTE ALTO,SP


In [7]:
cnpjs_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 930 entries, 0 to 993
Data columns (total 4 columns):
name     930 non-null object
cnpj     930 non-null object
city     930 non-null object
state    930 non-null object
dtypes: object(4)
memory usage: 36.3+ KB


# Merging Data

## GS1 <> CNPJs

In [8]:
    gs1 = pd.read_csv(treated_data / 'gs1_treated.csv')
    cnpjs = pd.read_csv(treated_data / 'cnpjs_treated.csv')
        
    combined_data = pd.merge(left=gs1, right=cnpjs, how='left', on=['cnpj','state','city'])

In [9]:
cnpjs.head()

,name,cnpj,city,state
0,FROOTY COMERCIO E INDUSTRIA DE ALIMENTOS S.A.,68093095000179,ATIBAIA,SP
1,ESTRELLA DE GALICIA IMPORTACAO E COMERCIALIZAC...,13492669000190,SAO PAULO,SP
2,DROGARIA NOVA SABARA LTDA,74683392000177,SAO PAULO,SP
3,DJ INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,12423658000195,CARPINA,PE
4,FUGINI ALIMENTOS LTDA,588458000103,MONTE ALTO,SP


In [10]:
gs1.head()

,cnpj,gtin,city,state
0,60409075000152,7891000258491,SAO PAULO,SP
1,54360656001701,7898142863118,CAMPINAS,SP
2,14998371000119,7896005213018,FORTALEZA,CE
3,60409075000152,7891000107508,SAO PAULO,SP
4,4429377000111,7898292888436,SAO PAULO,SP


In [11]:
combined_data = combined_data[combined_data['name'].notnull()]

In [12]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7462 entries, 0 to 9803
Data columns (total 5 columns):
cnpj     7462 non-null int64
gtin     7462 non-null int64
city     7462 non-null object
state    7462 non-null object
name     7462 non-null object
dtypes: int64(2), object(3)
memory usage: 349.8+ KB


In [13]:
combined_data.head()

,cnpj,gtin,city,state,name
0,60409075000152,7891000258491,SAO PAULO,SP,NESTLE BRASIL LTDA.
1,60409075000152,7891000258491,SAO PAULO,SP,NESTLE BRASIL LTDA.
3,14998371000119,7896005213018,FORTALEZA,CE,J MACEDO S/A
4,60409075000152,7891000107508,SAO PAULO,SP,NESTLE BRASIL LTDA.
5,60409075000152,7891000107508,SAO PAULO,SP,NESTLE BRASIL LTDA.


In [14]:
combined_data.to_csv(treated_data/'valid_gtins.csv', index=False)

## Valid GTINs <> Cosmos

In [15]:
cosmos = pd.read_csv(treated_data/'cosmos_treated.csv')
cosmos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9713 entries, 0 to 9712
Data columns (total 2 columns):
gtin           9713 non-null int64
description    9672 non-null object
dtypes: int64(1), object(1)
memory usage: 151.8+ KB


In [16]:
valid_gtins = pd.read_csv(treated_data/'valid_gtins.csv')
valid_gtins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7462 entries, 0 to 7461
Data columns (total 5 columns):
cnpj     7462 non-null int64
gtin     7462 non-null int64
city     7462 non-null object
state    7462 non-null object
name     7462 non-null object
dtypes: int64(2), object(3)
memory usage: 291.6+ KB


In [17]:
gtin_data = pd.merge(left=valid_gtins, right=cosmos, on='gtin', how='left')
gtin_data = gtin_data[gtin_data['description'].notnull()]
gtin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6022 entries, 2 to 7521
Data columns (total 6 columns):
cnpj           6022 non-null int64
gtin           6022 non-null int64
city           6022 non-null object
state          6022 non-null object
name           6022 non-null object
description    6022 non-null object
dtypes: int64(2), object(4)
memory usage: 329.3+ KB


In [18]:
gtin_data.to_csv(treated_data/'gtin_data.csv', index=False)

## GTIN Data <> Infomix

In [19]:
infomix = pd.read_csv(treated_data/'infomix_treated.csv')
infomix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25844 entries, 0 to 25843
Data columns (total 3 columns):
cnpj        25844 non-null int64
gtin        25844 non-null int64
category    25844 non-null object
dtypes: int64(2), object(1)
memory usage: 605.8+ KB


In [20]:
gtin_data = pd.read_csv(treated_data/'gtin_data.csv')
gtin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 6 columns):
cnpj           6022 non-null int64
gtin           6022 non-null int64
city           6022 non-null object
state          6022 non-null object
name           6022 non-null object
description    6022 non-null object
dtypes: int64(2), object(4)
memory usage: 282.4+ KB


In [21]:
infomix.cnpj.map(str).apply(len).max()

14

In [22]:
gtin_data.cnpj.map(str).apply(len).max()

14

In [23]:
final_df = pd.merge(infomix, gtin_data, how='inner',on='gtin')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15056 entries, 0 to 15055
Data columns (total 8 columns):
cnpj_x         15056 non-null int64
gtin           15056 non-null int64
category       15056 non-null object
cnpj_y         15056 non-null int64
city           15056 non-null object
state          15056 non-null object
name           15056 non-null object
description    15056 non-null object
dtypes: int64(3), object(5)
memory usage: 1.0+ MB


In [24]:
final_df = final_df.rename(columns={'cnpj_x':'cnpj_store', 'cnpj_y':'cnpj_manufacturer'})

In [25]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15056 entries, 0 to 15055
Data columns (total 8 columns):
cnpj_store           15056 non-null int64
gtin                 15056 non-null int64
category             15056 non-null object
cnpj_manufacturer    15056 non-null int64
city                 15056 non-null object
state                15056 non-null object
name                 15056 non-null object
description          15056 non-null object
dtypes: int64(3), object(5)
memory usage: 1.0+ MB


In [26]:
final_df.to_csv(treated_data/'final_result.csv', index=False)

# DONE!